### 腳位定義
- LED (D4)
- Buzzer (D2)
- T/H sensor (D1)
- 光遮斷器 (D0)
- PIR (D5)
- 超音波sensor (D7, D8)-->(Trig, Echo)
- 注意!! (D3)接buzzer上電後會有問題

In [ ]:
%websocketconnect --password 1234 ws://192.168.43.245:8266

In [2]:
%serialconnect to --port=com13 --baud=115200

Connecting to --port=com13 --baud=115200 
Ready.


In [3]:
# 定義
import machine
from machine import Pin
from machine import PWM
import time

# PIN Define:
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM


## Led

In [4]:
# Led ON/OFF 3次
from machine import Pin
import time

p2 = Pin(2, Pin.OUT) #D4
p2.value(1)
for i in range(6):
    p2.value(not p2.value())
    time.sleep(1)

.

## 光遮斷器

In [5]:
from machine import Pin
import time

p0 = Pin(D0, Pin.IN)

for i in range(6):
    print(p0.value())
    time.sleep(1)

0
0
1
0
0
1


## PIR

In [6]:
from machine import Pin
import time

p5 = Pin(D5, Pin.IN)
time.sleep(60) #初始化要60秒!?
print('start!')

for i in range(30):
    print(p5.value())
    time.sleep(1)

.

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 5, in <module>
KeyboardInterrupt: 


## NeoPixels (WS2812B)


In [7]:
import machine, neopixel
np = neopixel.NeoPixel(machine.Pin(D6), 8)
np[0] = (32, 0, 0) # set to red #255為全亮
np[1] = (0, 16, 0) # set to green, half brightness
np[2] = (0, 0, 8)  # set to blue, quarter brightness
np.write()

In [8]:
for i in range(32):
    for j in range(8):
        k = (j+(i))%3
        if(k==0):
            np[j] = (i, 0, 0)
        if(k==1):
            np[j] = (0, i, 0)
        if(k==2):
            np[j] = (0, 0, i)
    np.write()
    time.sleep(0.5)

...

In [9]:
for i in range(32):
    for j in range(8):
        k = (j+(i))%8
        if(k==0):
            np[j] = (i, 0, 0)
        if(k==1):
            np[j] = (0, i, 0)
        if(k==2):
            np[j] = (0, 0, i)
        if(k==3):
            np[j] = (i, i, 0)
        if(k==4):
            np[j] = (0, i, i)
        if(k==5):
            np[j] = (i, 0, i)
        if(k==6):
            np[j] = (i, i, i)
        if(k==7):
            np[j] = (0, 0, 0)
    np.write()
    time.sleep(0.2)

.

In [10]:
# 全暗
for i in range(8):
    np[i] = (0, 0, 0)
np.write()

In [11]:
import time

def demo(np):
    n = np.n

    # cycle
    for i in range(4 * n):
        for j in range(n):
            np[j] = (0, 0, 0)
        np[i % n] = (255, 255, 255)
        np.write()
        time.sleep_ms(25)

    # bounce
    for i in range(4 * n):
        for j in range(n):
            np[j] = (0, 0, 128)
        if (i // n) % 2 == 0:
            np[i % n] = (0, 0, 0)
        else:
            np[n - 1 - (i % n)] = (0, 0, 0)
        np.write()
        time.sleep_ms(60)

    # fade in/out
    for i in range(0, 4 * 256, 8):
        for j in range(n):
            if (i // 256) % 2 == 0:
                val = i & 0xff
            else:
                val = 255 - (i & 0xff)
            np[j] = (val, 0, 0)
        np.write()

    # clear
    for i in range(n):
        np[i] = (0, 0, 0)
    np.write()

demo(np)    

.

## 蜂鳴器

- 有源 vs 無源? (active vs passive)
- 本課程使用的為「無源蜂鳴器」 (passive buzzer)
- 腳位(D3 -> GPIO0)

*** ESP8266 的 PWM 最高只能到 1000 Hz, C6 (1046 Hz) 以上的聲音是無法發出來的

In [12]:
# 電話鈴聲

from machine import Pin,PWM
import time

def ringTone(pwm):
    for i in range(1,11):
        pwm.freq(1000)
        pwm.duty(512)
        time.sleep_ms(50)
        pwm.freq(500)
        time.sleep_ms(50)
    pwm.deinit()
    time.sleep(2)

pwm=PWM(Pin(4)) #D2

for i in range(4):
    ringTone(pwm)

..

In [34]:
# 暖暖，前半部

from machine import Pin,PWM
import time

C4=262
CS4=277
D4=294
DS4=311
E4=330
F4=349
FS4=370
G4=392
GS4=415
A4=440
AS4=466
B4=494
C5=523
#CS5=554
D5=587
#DS5=622
E5=659
F5=698
#FS5=740
G5=784
#GS5=831
A5=880
#AS5=932
B5=988

note=(G4, C5, D5, D5, E5, E5, D5, E5, E5, 
      D5, E5, G5, E5, C5, C5, G4, A4, E5, D5, C5, D5,
      E5, 0 , G4, C5, D5, D5, E5, E5, D5, E5, E5, 
      D5, E5, G5, E5, C5, C5, B4, A4, E5, D5, C5, D5, 
      C5,
      )
duration=(8, 8, 8, 8, 8, 4, 8, 8, 4, 
          8, 8, 8, 8, 4, 8, 8, 3, 8, 4, 8, 8, 
          2, 8, 8, 8, 8, 8, 8, 4, 8, 8, 4,
          8, 8, 8, 8, 4, 8, 8, 3, 8, 4, 8, 8, 
          1,
          )

def tone(pwm,note,duration):
    if note>0:
        pwm.freq(note)           
        pwm.duty(512)
    time.sleep_ms(duration)
    pwm.deinit()
    time.sleep_ms(int(duration/4))

def music(pwm):
    for i in range(0,len(note)):
        d=int(1000/duration[i])
        tone(pwm, note[i], d*2)
        #p=int(d*1.3)
        #time.sleep_ms(p)
    time.sleep(1)

#pwm=PWM(Pin(0)) #D3
pwm=PWM(Pin(4)) #D2

for i in range(1):
    music(pwm)

....

## 溫溼度感測器


In [13]:
# upy to read temperature, humidity

import dht
from machine import Pin

d = dht.DHT11(Pin(5)) #D1
d.measure()
T = d.temperature() # eg. 23 (℃)
H = d.humidity()    # eg. 41 (% RH)
print("T=%s, H=%s" %(T, H))


T=29, H=95


## 超音波

- HC-SR04

In [14]:
# 定義
from machine import Pin
from machine import PWM
import time

# PIN Define:
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM


In [15]:
# 340m/s --> 340000mm/s --> 1mm需要2.94 us
# 340m/s --> 34000cm/s --> 1cm要29.4us --> 去回，所以除以(29.4*2)=58.8，但手冊是寫58us為1cm
# 所以如果目標是50cm內的量測距離 --> 50*58=2900us為最長時間
# timeout回傳0，很怪，應該給個大數字比較好，比如999

import time
from machine import Pin

def ping(trigPin, echoPin):
    '''
        return: distance (cm)
    '''
    trig=Pin(trigPin, Pin.OUT)
    echo=Pin(echoPin, Pin.IN)
    trig.value(1)
    time.sleep_us(10)
    trig.value(0)
    timeout=False
    tm_start=time.ticks_us() 
    while not echo.value(): #wait for HIGH, 3000us timeout
        if(time.ticks_diff(time.ticks_us(), tm_start)>3000):
            timeout=True
            break
    if timeout: #timeout return 0
        pass
    else: #got HIGH pulse:calculate duration
        tm_start=time.ticks_us()
        tm_delta = 0
        while echo.value(): #wait for LOW
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            if(tm_delta>3000):
                timeout=True
                break
        if timeout:
            pass
        else:
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            duration=tm_delta
    
    if timeout:
        return 999 #cm, for timeout
        
    return duration/58

while True:
    distance=ping(trigPin=D7,echoPin=D8)
    print('%s cm' % distance)
    time.sleep(1)

999 cm
999 cm
999 cm
.50.9828 cm
10.1379 cm
12.2069 cm
10.3793 cm
10.1379 cm


*** Sending Ctrl-C

.37931 cm


Traceback (most recent call last):
  File "<stdin>", line 48, in <module>
KeyboardInterrupt: 


## 上傳thingSpeak

- 帳號key


In [ ]:
%serialconnect to --port=com4 --baud=115200

In [37]:
%websocketconnect --password 1234 ws://192.168.43.106:8266

** WebSocket connected **
Password: 
WebREPL connected
>>> Ready.


In [42]:
# test
import gc
res = gc.collect()
print(res)

res = gc.mem_free()
print(res)

import micropython
res = micropython.mem_info()
print(res)

None
22800
stack: 2096 out of 8192
GC: total: 36288, used: 13488, free: 22800
 No. of 1-blocks: 91, 2-blocks: 11, max blk sz: 264, max free sz: 910
None


In [36]:
#%sendtofile main.py --binary

import urequests
import dht
from machine import Pin

url = "http://api.thingspeak.com/update?api_key=TWB7BD496GOYPGM1&field1=25&field2=65"
api_key = 'TWB7BD496GOYPGM1'
field1 = 20 #T
field2 = 60 #H

d = dht.DHT11(Pin(D1))

while(True):
    try:
        d.measure()
        T = d.temperature() # eg. 23 (℃)
        H = d.humidity()    # eg. 41 (% RH)
        print("T=%s, H=%s" %(T, H))

        url_update = 'http://api.thingspeak.com/update?api_key='+api_key+'&field1='+str(T)+'&field2='+str(H)
        r = urequests.post(url_update)
        print('result: ', r.content)
        r.close()
    except Exception as e:
        print('error:', str(e))
        

T=32, H=95
.error: -2
T=32, H=94
..error: -2
T=32, H=94


*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 26, in <module>
  File "<stdin>", line 22, in <module>
  File "urequests.py", line 104, in post
  File "urequests.py", line 51, in request
KeyboardInterrupt: 


## web tool

In [31]:
%serialconnect to --port=com13 --baud=115200

Connecting to --port=com13 --baud=115200 
Ready.


- 燒錄AP設定檔的方法(1)

In [32]:
%sendtofile ap.txt

malo-ap
0928380233

Sent 2 lines (18 bytes) to ap.txt.


- 燒錄AP設定檔的方法(2)

In [33]:
ap_config_fn = 'ap.txt'

In [ ]:
AP_SSID = 'my-ap-name'
AP_PWD = 'my-password'
with open(ap_config_fn, 'w') as f:
    f.write(AP_SSID+'\n')
    f.write(AP_PWD+'\n')

In [34]:
ap_config = None

print('ap config here!')
with open(ap_config_fn, 'r') as f:
    ap_config = f.read()

print(ap_config)
ap_config = ap_config.split('\n')
AP_SSID = ap_config[0].strip()
AP_PWD = ap_config[1].strip()
print(ap_config)

ap config here!
malo-ap
0928380233
['malo-ap', '0928380233']


- 燒錄main.py主程式 & 網頁介面

In [10]:
%sendtofile main.py --binary

import machine
import network
import time
import os

#- check ap config file
AP_SSID = 'upy'
AP_PWD = 'pypypypy'
ap_config = None
ap_config_fn = 'ap.txt'
if ap_config_fn in os.listdir():
    print('ap config here!')
    f = open(ap_config_fn)
    ap_config = f.read()
    f.close()
if ap_config:
    print( ('ap_config:', ap_config))
    ap_config = ap_config.split('\n')
    AP_SSID = ap_config[0].strip()
    AP_PWD = ap_config[1].strip()
print('line to: ', (AP_SSID, AP_PWD))

#-- 連到AP 為Station
sta_if = network.WLAN(network.STA_IF)
sta_if.active(True)
sta_if.connect(AP_SSID, AP_PWD)
sta_if.isconnected()
for i in range(20):
    time.sleep(0.5)
    if sta_if.isconnected():
        break
sta_ip = ''
if sta_if.isconnected():
    print('connected!  --> ', sta_if.ifconfig())
    sta_ip = sta_if.ifconfig()[0]
else:
    print('not connected!  --> ', sta_if.ifconfig())

#-- 當AP，並指定
uid = machine.unique_id()
#ap_if.ifconfig()
# ('192.168.4.1', '255.255.255.0', '192.168.4.1', '192.168.43.1')
# (ip, mask, gateway, dns)
my_sn = '%02X-%02X-%02X' %(uid[0], uid[1], uid[2])

#- Change name/password/ip of ESP8266's AP:
ap_if = network.WLAN(network.AP_IF)
#ap_if.ifconfig([my_ip, my_mask, my_gw, my_dns])

my_ssid = 'up_%s' %(sta_ip)
#ap_if.config(essid = my_ssid)#改ssid，馬上生效
ap_if.config(essid=my_ssid, authmode=network.AUTH_WPA_WPA2_PSK, password="12345678")
#DHCP 功能micropython預設就有，不用設定
#AP的IP，每次重開都會回到預設值，因此需要開機時就設定
#一般是配給AP ip的下一號ip


import socket
from machine import Pin
from machine import PWM
import dht
#from hcsr04 import HCSR04

def Tune(buzzer, freq=262, t=500, duty=512):
    ''' t: ms '''
    if freq==0:
        buzzer.duty(0)
    else:
        buzzer.duty(duty)
    buzzer.freq(freq)
    time.sleep_ms(t)


# PIN Define:
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM

#Setup PINS
led = machine.Pin(D4, machine.Pin.OUT)

# buzzer
NOTE_C4=262
NOTE_D4=294
NOTE_E4=330
NOTE_F4=349
NOTE_G4=392
NOTE_A4=440
NOTE_B4=494
buzzer = PWM(Pin(D2), freq=1000, duty=0)

# servo
SV_ANGLE_0 = 40
SV_ANGLE_90 = 77
SV_ANGLE_180 = 115
servo = PWM(Pin(D5), freq=50)
servo2 = PWM(Pin(D6), freq=50)
servo3 = PWM(Pin(D7), freq=50)
servo4 = PWM(Pin(D8), freq=50)

# th_sensor
th_sensor = dht.DHT11(Pin(D1))

# HCSR04, not work, TODO: later
#sr04 = HCSR04(trigger_pin=D7, echo_pin=D8)

#Setup Socket WebServer
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind(('0.0.0.0', 80))
s.listen(5)
while True:
    conn, addr = s.accept()
    print("Got a connection from %s" % str(addr))
    request = conn.recv(1024)
    print("Content = %s" % str(request))
    '''
    Got a connection from ('10.107.85.22', 64869)
    Content = b'GET /favicon.ico HTTP/1.1\r\nHost: 10.107.85.21\r\nConnection: keep-alive\r\nUser-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36\r\nAccept: image/webp,image/apng,image/*,*/*;q=0.8\r\nReferer: http://10.107.85.21/\r\nAccept-Encoding: gzip, deflate\r\nAccept-Language: zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7\r\n\r\n'
    '''
    request = str(request)
    led_on = request.find('GET /?LED=ON')
    led_off = request.find('GET /?LED=OFF')
    buzzer_on = request.find('GET /?buzzer=on')
    buzzer_off = request.find('GET /?buzzer=off')
    th_sensor_read = request.find('GET /?th_sensor=read')
    sr04_read = request.find('GET /?sr04=read')


    if led_on >= 0:
        print('TURN Led ON')
        led.value(0)
    if led_off >= 0:
        print('TURN Led OFF')
        led.value(1)
    if buzzer_on >= 0:
        print('buzzer on')
        Tune(buzzer, NOTE_C4, 10)
        time.sleep(4)
        Tune(buzzer, 0, 10)
    if buzzer_off >= 0:
        print('buzzer off')
        Tune(buzzer, 0, 10)

    my_t = None
    my_h = None
    if th_sensor_read >= 0:
        print('th_sensor read')
        th_sensor.measure()
        my_t = th_sensor.temperature()
        my_h = th_sensor.humidity()
        print('T=', my_t)
        print('H=', my_h)
    my_distance = None
    if sr04_read >= 0:
        print('sr04 read')
        my_distance = -1 #sr04.distance_cm()
        print('distance=', my_distance)

    f = open('webtool.html')

    while(1):
        html = f.read(1024)

        if my_t:
            html = html.replace('T=?degree', 'T=%d degree' %(my_t))
        else:
            html = html.replace('T=?degree', '')
            
        if my_h:
            html = html.replace('H=?%', 'T=%d %%' %(my_h))
        else:
            html = html.replace('H=?%', '')
            
        if my_distance:
            html = html.replace('distance=? cm', 'distance=%s cm' %(my_distance))
        else:
            html = html.replace('distance=? cm', '')

        #conn.send(html)
        conn.sendall(html) #改用send all就不會有資料傳一半的問題
        if(len(html)<=0):
            break
    f.close()
    conn.close()


Sent 5031 bytes in 168 chunks to main.py.


In [11]:
%sendtofile webtool.html --binary

<!DOCTYPE html>
<html>
<head> <title>upy webserver</title> </head>
<center><h2>upy webserver</h2></center>
<h3>
webREPL: <a href="http://micropython.org/webrepl/" target="_blank">webrepl</a>
<br>
Doc: <a href="https://docs.micropython.org/en/latest/esp8266/esp8266/quickref.html" target="_blank">Quick reference</a>
</h3>
<hr>

<form>
<b>LED (D4):</b><br>
<button name="LED" value="ON" type="submit">ON</button><br>
<button name="LED" value="OFF" type="submit">OFF</button><br>
</form>

<form>
<b>Buzzer (D2):</b><br>
<button name="buzzer" value="on" type="submit">ON</button><br>
<button name="buzzer" value="off" type="submit">OFF</button><br>
</form>

<form>
<b>T/H sensor (D1):</b><br>
<button name="th_sensor" value="read" type="submit">Read</button><br>
T=?degree
H=?%
</form>

<hr>
GPIO Info. :<br>
<pre>
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM
</pre>

</html>

Sent 954 bytes in 32 chunks to webtool.html.


### 燒錄main.py & 網頁介面 (加入超音波)

In [35]:
%sendtofile main.py --binary

import machine
import network
import time
import os

#- check ap config file
AP_SSID = 'upy'
AP_PWD = 'pypypypy'
ap_config = None
ap_config_fn = 'ap.txt'
if ap_config_fn in os.listdir():
    print('ap config here!')
    f = open(ap_config_fn)
    ap_config = f.read()
    f.close()
if ap_config:
    print( ('ap_config:', ap_config))
    ap_config = ap_config.split('\n')
    AP_SSID = ap_config[0].strip()
    AP_PWD = ap_config[1].strip()
print('line to: ', (AP_SSID, AP_PWD))

#-- 連到AP 為Station
sta_if = network.WLAN(network.STA_IF)
sta_if.active(True)
sta_if.connect(AP_SSID, AP_PWD)
sta_if.isconnected()
for i in range(20):
    time.sleep(0.5)
    if sta_if.isconnected():
        break
sta_ip = ''
if sta_if.isconnected():
    print('connected!  --> ', sta_if.ifconfig())
    sta_ip = sta_if.ifconfig()[0]
else:
    print('not connected!  --> ', sta_if.ifconfig())

#-- 當AP，並指定
uid = machine.unique_id()
#ap_if.ifconfig()
# ('192.168.4.1', '255.255.255.0', '192.168.4.1', '192.168.43.1')
# (ip, mask, gateway, dns)
my_sn = '%02X-%02X-%02X' %(uid[0], uid[1], uid[2])

#- Change name/password/ip of ESP8266's AP:
ap_if = network.WLAN(network.AP_IF)
#ap_if.ifconfig([my_ip, my_mask, my_gw, my_dns])

my_ssid = 'up_%s' %(sta_ip)
#ap_if.config(essid = my_ssid)#改ssid，馬上生效
ap_if.config(essid=my_ssid, authmode=network.AUTH_WPA_WPA2_PSK, password="12345678")
#DHCP 功能micropython預設就有，不用設定
#AP的IP，每次重開都會回到預設值，因此需要開機時就設定
#一般是配給AP ip的下一號ip


import socket
from machine import Pin
from machine import PWM
import dht
#from hcsr04 import HCSR04

def Tune(buzzer, freq=262, t=500, duty=512):
    ''' t: ms '''
    if freq==0:
        buzzer.duty(0)
    else:
        buzzer.duty(duty)
    buzzer.freq(freq)
    time.sleep_ms(t)

# 340m/s --> 340000mm/s --> 1mm需要2.94 us
# 340m/s --> 34000cm/s --> 1cm要29.4us --> 去回，所以除以(29.4*2)=58.8，但手冊是寫58us為1cm
# 所以如果目標是50cm內的量測距離 --> 50*58=2900us為最長時間
# timeout回傳0，很怪，應該給個大數字比較好，比如999

import time
from machine import Pin

def ping(trigPin, echoPin):
    '''
        return: distance (cm)
    '''
    trig=Pin(trigPin, Pin.OUT)
    echo=Pin(echoPin, Pin.IN)
    trig.value(1)
    time.sleep_us(10)
    trig.value(0)
    timeout=False
    tm_start=time.ticks_us() 
    while not echo.value(): #wait for HIGH, 3000us timeout
        if(time.ticks_diff(time.ticks_us(), tm_start)>3000):
            timeout=True
            break
    if timeout: #timeout return 0
        pass
    else: #got HIGH pulse:calculate duration
        tm_start=time.ticks_us()
        tm_delta = 0
        while echo.value(): #wait for LOW
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            if(tm_delta>3000):
                timeout=True
                break
        if timeout:
            pass
        else:
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            duration=tm_delta
    
    if timeout:
        return 999 #cm, for timeout
        
    return duration/58

    
# PIN Define:
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM

#Setup PINS
led = machine.Pin(D4, machine.Pin.OUT)

# buzzer
NOTE_C4=262
NOTE_D4=294
NOTE_E4=330
NOTE_F4=349
NOTE_G4=392
NOTE_A4=440
NOTE_B4=494
buzzer = PWM(Pin(D2), freq=1000, duty=0)

# th_sensor
th_sensor = dht.DHT11(Pin(D1))

#Setup Socket WebServer
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind(('0.0.0.0', 80))
s.listen(5)
while True:
    conn, addr = s.accept()
    print("Got a connection from %s" % str(addr))
    request = conn.recv(1024)
    print("Content = %s" % str(request))
    '''
    Got a connection from ('10.107.85.22', 64869)
    Content = b'GET /favicon.ico HTTP/1.1\r\nHost: 10.107.85.21\r\nConnection: keep-alive\r\nUser-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36\r\nAccept: image/webp,image/apng,image/*,*/*;q=0.8\r\nReferer: http://10.107.85.21/\r\nAccept-Encoding: gzip, deflate\r\nAccept-Language: zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7\r\n\r\n'
    '''
    request = str(request)
    led_on = request.find('GET /?LED=ON')
    led_off = request.find('GET /?LED=OFF')
    buzzer_on = request.find('GET /?buzzer=on')
    buzzer_off = request.find('GET /?buzzer=off')
    th_sensor_read = request.find('GET /?th_sensor=read')
    sr04_read = request.find('GET /?sr04=read')


    if led_on >= 0:
        print('TURN Led ON')
        led.value(0)
    if led_off >= 0:
        print('TURN Led OFF')
        led.value(1)
    if buzzer_on >= 0:
        print('buzzer on')
        Tune(buzzer, NOTE_C4, 10)
        time.sleep(4)
        Tune(buzzer, 0, 10)
    if buzzer_off >= 0:
        print('buzzer off')
        Tune(buzzer, 0, 10)

    my_t = None
    my_h = None
    if th_sensor_read >= 0:
        print('th_sensor read')
        th_sensor.measure()
        my_t = th_sensor.temperature()
        my_h = th_sensor.humidity()
        print('T=', my_t)
        print('H=', my_h)
    my_distance = None
    if sr04_read >= 0:
        print('sr04 read')
        my_distance=ping(trigPin=D7,echoPin=D8)
        print('distance=', my_distance)

    f = open('webtool.html')

    while(1):
        html = f.read(1024)

        if my_t:
            html = html.replace('T=?degree', 'T=%d degree' %(my_t))
        else:
            html = html.replace('T=?degree', '')
            
        if my_h:
            html = html.replace('H=?%', 'T=%d %%' %(my_h))
        else:
            html = html.replace('H=?%', '')
            
        if my_distance:
            html = html.replace('distance=? cm', 'distance=%s cm' %(my_distance))
        else:
            html = html.replace('distance=? cm', '')

        #conn.send(html)
        conn.sendall(html) #改用send all就不會有資料傳一半的問題
        if(len(html)<=0):
            break
    f.close()
    conn.close()


Sent 6093 bytes in 204 chunks to main.py.


In [36]:
%sendtofile webtool.html --binary

<!DOCTYPE html>
<html>
<head> <title>upy webserver</title> </head>
<center><h2>upy webserver</h2></center>
<h3>
webREPL: <a href="http://micropython.org/webrepl/" target="_blank">webrepl</a>
<br>
Doc: <a href="https://docs.micropython.org/en/latest/esp8266/esp8266/quickref.html" target="_blank">Quick reference</a>
</h3>
<hr>

<form>
<b>LED (D4):</b><br>
<button name="LED" value="ON" type="submit">ON</button><br>
<button name="LED" value="OFF" type="submit">OFF</button><br>
</form>

<form>
<b>Buzzer (D2):</b><br>
<button name="buzzer" value="on" type="submit">ON</button><br>
<button name="buzzer" value="off" type="submit">OFF</button><br>
</form>

<form>
<b>T/H sensor (D1):</b><br>
<button name="th_sensor" value="read" type="submit">Read</button><br>
T=?degree
H=?%
</form>

<form>
<b>ultra sonic (D7,8):</b><br>
<button name="sr04" value="read" type="submit">Read</button><br>
distance=? cm
</form>

<hr>
GPIO Info. :<br>
<pre>
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM
</pre>

</html>

Sent 1080 bytes in 37 chunks to webtool.html.


## scan wifi

In [34]:
%serialconnect to --port=com4 --baud=115200

Connecting to --port=com4 --baud=115200 
Ready.


In [20]:
sta_if = network.WLAN(network.STA_IF)
sta_if.active(True)
result = sta_if.scan()
print(result)
'''
result:

[(b'DELTA_11NG', b'\x00\x18#\x01>h', 6, -82, 0, 0), (b'malo-ap', b'\xc4\x0b\xcb8+_', 6, -51, 3, 0), (b'ICPDAS-EE3', b'0Z:`\xc7\x00', 3, -54, 4, 0), (b
'ICP DAS', b'\x10o?\xdae\xf6', 6, -71, 4, 0), (b'Staff', b'\x00P\x18!\xe7\x87', 11, -72, 0, 0), (b'ICPDAS-UNIFI', b'\n\x18\xd6%+7', 11, -71, 3, 0), (b
'WM05B_QC', b'\xf4^\xab~?n', 11, -89, 4, 0), (b'Alex_test', b'\x00P\x18!\xe5\xc9', 11, -48, 3, 0), (b'WF-2571', b'\x02P\x18!\xe5\xc9', 11, -48, 3, 0),
 (b'611212', b'\x00\xd0A\xd6S\xd9', 11, -91, 3, 0), (b'EDIMAX', b'\x80\x1f\x02\xb7\xde\xd8', 11, -91, 3, 0)]    
'''

[(b'P874', b'\xc8l\x876\xbcy', 1, -75, 2, 0), (b'CHT0095', b"\xd8\xfe\xe3[\xe8'", 1, -90, 3, 0), (b'SEALER', b'\x00\x16\x01Xj]', 5, -92, 1, 0), (b'malo-ap', b'\xc4\x0b\xcb8+_', 6, -54, 3, 0), (b'.1.Free Wi-Fi', b'pb\xb8\xb7\xd6\x93', 6, -89, 0, 0), (b'CHT Wi-Fi(HiNet)', b'pb\xb8\xb7\xd6\x91', 6, -86, 0, 0), (b'MOTOROLA-A64B2', b'\xf85\xdd]\x9eW', 8, -65, 3, 0), (b'CHT7381', b'@\x9b\xcd\x02\x9a\xea', 11, -70, 3, 0), (b'6740C', b'\xcc\xb2U\xff\xa89', 11, -83, 3, 0), (b'LIANSHEN', b'\\\xf4\xabn\x80\x1c', 11, -90, 3, 0)]


In [23]:
for i in result:
    print(i)

(b'P874', b'\xc8l\x876\xbcy', 1, -75, 2, 0)
(b'CHT0095', b"\xd8\xfe\xe3[\xe8'", 1, -90, 3, 0)
(b'SEALER', b'\x00\x16\x01Xj]', 5, -92, 1, 0)
(b'malo-ap', b'\xc4\x0b\xcb8+_', 6, -54, 3, 0)
(b'.1.Free Wi-Fi', b'pb\xb8\xb7\xd6\x93', 6, -89, 0, 0)
(b'CHT Wi-Fi(HiNet)', b'pb\xb8\xb7\xd6\x91', 6, -86, 0, 0)
(b'MOTOROLA-A64B2', b'\xf85\xdd]\x9eW', 8, -65, 3, 0)
(b'CHT7381', b'@\x9b\xcd\x02\x9a\xea', 11, -70, 3, 0)
(b'6740C', b'\xcc\xb2U\xff\xa89', 11, -83, 3, 0)
(b'LIANSHEN', b'\\\xf4\xabn\x80\x1c', 11, -90, 3, 0)
